# Simulations
In this iPython notebook, we run the double auction version of the model. 

In [1]:
import pandas as pd
import numpy as np
from ipywidgets import widgets, interact
from stockmarket import baselineCDA
import sqlite3
from tqdm import *

In [2]:
from stockmarket.database import create_tables

In [28]:
NRUNS = 1
EXPERIMENT_ID = 1
STARTING_PERIOD = 0
SEED = 0
SIMULATIONTIME = 100

In [4]:
conn = sqlite3.connect('dataBaseCDAuction.sqlite')
cur = conn.cursor()

In [5]:
for seed in range(NRUNS):
    agents, firms, stocks, order_books = baselineCDA.stockMarketSimulation(seed=seed, simulation_time=SIMULATIONTIME,
                                                                           amount_fundamentalists=40, 
                                                                           amount_chartists=40, amount_firms=1,
                                                                           initial_money=(100, 200), 
                                                                           initial_bid_ask=(1,1), 
                                                                           initial_memory=(3,7),
                                                                           initial_ma_short=(1,3), 
                                                                           initial_ma_long=(6,7), 
                                                                           initial_profit=(200,200),
                                                                           initial_book_value=(10000, 10000), 
                                                                           initial_stock_amount=400, 
                                                                           order_expiration_time=120
                                                                          )
#     Transactions.to_sql("Transactions{}".format(seed), conn, if_exists="replace")
#     Transactors.to_sql("Transactors{}".format(seed), conn, if_exists="replace")
#     Statevariables.to_sql("Statevariables{}".format(seed), conn, if_exists="replace")
#     Variabletypes.to_sql("Variabletypes{}".format(seed), conn, if_exists="replace")
#     Objects.to_sql("Objects{}".format(seed), conn, if_exists="replace")

In [49]:
create_tables(cur)

In [29]:
# add agents variables to the SQL database 
for agent in agents:
    # 1 store the agent and type of agent in the objects table
    cur.execute("INSERT OR IGNORE INTO Objects (object_name, object_type) VALUES (?,?)", (repr(agent), repr(agent)[:repr(agent).find('_')]))
    cur.execute("SELECT id FROM Objects WHERE object_name = ?", (repr(agent),))
    owner_id = cur.fetchone()[0]
    # for the agent-variables
    variables = vars(agent)
    for variable in variables:
        # store the variable type in the variable types table
        cur.execute("INSERT OR IGNORE INTO Variabletypes (variable_type) VALUES (?)", (str(variable), ))
        cur.execute("SELECT id FROM Variabletypes WHERE variable_type = ?", (str(variable),))
        variable_type_id = cur.fetchone()[0]
        # store the variable content in the state variables table 
        if not 'history' in str(variable):
            cur.execute("INSERT INTO Statevariables (experiment_id, seed, period, variable_id, owner_id, value) VALUES (?,?,?,?,?,?)",
                        (EXPERIMENT_ID, SEED, STARTING_PERIOD, variable_type_id, owner_id, str(variables[variable])))
        else:
            if len(variables[variable]) > SIMULATIONTIME:
                var = variables[variable][(len(variables[variable])-SIMULATIONTIME):]
            else: 
                var = variables[variable]
            for idx, element in enumerate(var):
                cur.execute("INSERT INTO Statevariables (experiment_id, seed, period, variable_id, owner_id, value) VALUES (?,?,?,?,?,?)",
                            (EXPERIMENT_ID, SEED, idx, variable_type_id, owner_id, str(element)))

In [36]:
# add stock variables to the SQL database 
for stock in stocks:
    # 1 store the stock and type of stock in the objects table
    cur.execute("INSERT OR IGNORE INTO Objects (object_name, object_type) VALUES (?,?)", (repr(stock), repr(stock)[:repr(stock).find('_')]))
    cur.execute("SELECT id FROM Objects WHERE object_name = ?", (repr(stock),))
    owner_id = cur.fetchone()[0]
    # for the stock-variables
    variables = vars(stock)
    for variable in variables:
        # store the variable type in the variable types table
        cur.execute("INSERT OR IGNORE INTO Variabletypes (variable_type) VALUES (?)", (str(variable), ))
        cur.execute("SELECT id FROM Variabletypes WHERE variable_type = ?", (str(variable),))
        variable_type_id = cur.fetchone()[0]
        # store the variable content in the state variables table 
        if not 'history' in str(variable):
            cur.execute("INSERT INTO Statevariables (experiment_id, seed, period, variable_id, owner_id, value) VALUES (?,?,?,?,?,?)",
                        (EXPERIMENT_ID, SEED, STARTING_PERIOD, variable_type_id, owner_id, str(variables[variable])))
# do not append the price history from the stocks

In [50]:
for firm in firms:
    # 1 store the firm and type of firm in the objects table
    cur.execute("INSERT OR IGNORE INTO Objects (object_name, object_type) VALUES (?,?)", (repr(firm), repr(firm)[:repr(firm).find('_')]))
    cur.execute("SELECT id FROM Objects WHERE object_name = ?", (repr(firm),))
    owner_id = cur.fetchone()[0]
    # for the firm-variables
    variables = vars(firm)
    for variable in variables:
        # store the variable type in the variable types table
        cur.execute("INSERT OR IGNORE INTO Variabletypes (variable_type) VALUES (?)", (str(variable), ))
        cur.execute("SELECT id FROM Variabletypes WHERE variable_type = ?", (str(variable),))
        variable_type_id = cur.fetchone()[0]
        # store the variable content in the state variables table 
        if not 'history' in str(variable):
            cur.execute("INSERT INTO Statevariables (experiment_id, seed, period, variable_id, owner_id, value) VALUES (?,?,?,?,?,?)",
                        (EXPERIMENT_ID, SEED, STARTING_PERIOD, variable_type_id, owner_id, str(variables[variable])))
        else:
            if len(variables[variable]) > SIMULATIONTIME:
                var = variables[variable][(len(variables[variable])-SIMULATIONTIME):]
            else: 
                var = variables[variable]
            for idx, element in enumerate(var):
                cur.execute("INSERT INTO Statevariables (experiment_id, seed, period, variable_id, owner_id, value) VALUES (?,?,?,?,?,?)",
                            (EXPERIMENT_ID, SEED, idx, variable_type_id, owner_id, str(element)))

In [ ]:
for book in order_books:
    # 1 store the book and type of book in the objects table
    cur.execute("INSERT OR IGNORE INTO Objects (object_name, object_type) VALUES (?,?)", (repr(book), repr(book)[:repr(book).find('_')]))
    cur.execute("SELECT id FROM Objects WHERE object_name = ?", (repr(book),))
    owner_id = cur.fetchone()[0]
    # for the firm-variables
    variables = vars(book)
    for variable in variables:
        # store the variable type in the variable types table
        cur.execute("INSERT OR IGNORE INTO Variabletypes (variable_type) VALUES (?)", (str(variable), ))
        cur.execute("SELECT id FROM Variabletypes WHERE variable_type = ?", (str(variable),))
        variable_type_id = cur.fetchone()[0]
        # store the variable content in the state variables table 
        if not 'history' in str(variable):
            cur.execute("INSERT INTO Statevariables (experiment_id, seed, period, variable_id, owner_id, value) VALUES (?,?,?,?,?,?)",
                        (EXPERIMENT_ID, SEED, STARTING_PERIOD, variable_type_id, owner_id, str(variables[variable])))
        else:
            if len(variables[variable]) > SIMULATIONTIME:
                var = variables[variable][(len(variables[variable])-SIMULATIONTIME):]
            else: 
                var = variables[variable]
            for idx, element in enumerate(var):
                cur.execute("INSERT INTO Statevariables (experiment_id, seed, period, variable_id, owner_id, value) VALUES (?,?,?,?,?,?)",
                            (EXPERIMENT_ID, SEED, idx, variable_type_id, owner_id, str(element)))

In [51]:
conn.commit()

In [43]:
len(vars(firms[0])['profit_history'])-SIMULATIONTIME

7

In [35]:
len(vars(firms[0])['price_history'])

3130

In [12]:
conn.commit()
cur.close()